In [1]:
import sys
import os, os.path

sys.path.append(os.path.join(os.getcwd() ,'/modules'))
root_path = "C:/git/Springboard-Public/Capstone Project 2/"
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = "/content/drive/My Drive/Capstone Project 2/"

print('Current Working Dir: ', os.getcwd())
print('Root Path: ', root_path)

# We need to set the working directory since we are using relative paths from various locations
if os.getcwd() != root_path:
  os.chdir(root_path)

Current Working Dir:  C:\git\Springboard-Public\Capstone Project 2\notebooks\Support Notebooks for Modules
Root Path:  C:/git/Springboard-Public/Capstone Project 2/


In [2]:
import numpy as np
from datetime import datetime
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from modules.lib.ChextXRayImages import *
from modules.models.CustomPneumonia import CustomPneumoniaNN

from PIL import Image
import copy

import torch.optim as optim
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, ToPILImage
import torchvision.models as models

from torchsummary import summary

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

%matplotlib inline

In [3]:
force_cpu = True
device = torch.device('cuda' if ~force_cpu and torch.cuda.is_available() else 'cpu')
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(f'Working on device={device}')

Working on device=cuda


In [4]:
loaders = Loaders()
batch_size=16
val_percent=0.15
number_images = 1000
train_loader, val_loader = loaders.getDataTrainValidateLoaders(batch_size=batch_size, 
                                                                        val_percent=val_percent, 
                                                                        n_random_rows=number_images)
print(f'Number of Training Batches: {len(train_loader):,}')
print(f'Number of Validation Batches: {len(val_loader):,}')
print(f'Number of Training Images: {len(train_loader) * batch_size:,}')
print(f'Number of Validation Images: {len(val_loader) * batch_size:,}')

C:\git\Springboard-Public\Capstone Project 2\modules\lib\ChextXRayImages.py:244: UserWarning: 
Feature Imbalance Detected (train % - val %):
   Cardiomegaly: 2.12%
   Consolidation: 3.01%
   Pleural_Effusion: 3.98%

  self.warnFeatureImbalance(train, value)


Number of Training Batches: 55
Number of Validation Batches: 8
Number of Training Images: 880
Number of Validation Images: 128


In [5]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.softmax = nn.Softmax(dim=1)       
        self.flattened_length_ = 1*320*320
        self.fc1 = nn.Linear(self.flattened_length_, 12)
       
    def forward(self, x):    
        x = x.view(-1, self.flattened_length_)    
        x = self.fc1(x)
        return x

In [6]:
net = SimpleModel()

net = nn.DataParallel(net)
net.to(device)

summary(net, (1, 320, 320))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 12]       1,228,812
       SimpleModel-2                   [-1, 12]               0
Total params: 1,228,812
Trainable params: 1,228,812
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.39
Forward/backward pass size (MB): 0.00
Params size (MB): 4.69
Estimated Total Size (MB): 5.08
----------------------------------------------------------------


In [7]:
data = next(iter(train_loader))
ImageID, inputs, labels = data['id'], data['img'], data['labels']

print('Batch ImageIDs: ', ImageID.detach().numpy())

print(labels.shape)

# move data to device GPU OR CPU
inputs = inputs.to(device)
labels = labels.to(device)

outputs = net(inputs)
print(inputs.shape)
print(outputs.shape)

print('-' * 50)

predicted = outputs.data
predicted = torch.sigmoid(predicted) 

predicted[predicted >= 0.5] = 1 # assign 1 label to those with less than 0.5
predicted[predicted < 0.5] = 0 # assign 0 label to those with less than 0.5
print(predicted, '\n')
print(labels)


print('-' * 50)

train_batch_size, train_label_count = labels.shape

print('Accurate Predictions: ', (predicted == labels).sum())
print('Total Predictions: ', train_batch_size * train_label_count)
train_acc = float((predicted == labels).sum()) / float((train_batch_size * train_label_count))
print(train_acc)

Batch ImageIDs:  [ 90069 184464  12412  13573 138960 219437  51083 221958 148671 214194
  51529 222821 213681 118589    643 216996]
torch.Size([16, 12])
torch.Size([16, 1, 320, 320])
torch.Size([16, 12])
--------------------------------------------------
tensor([[0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1.],
        [1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1.],
        [1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 

In [8]:
def parseLoaderData(data):
    """
    The data loaders output a dictionary with 3 keys
    The first 2 keys hold single values for the ImageID and the actual tensor of the image
    The last key holds a vector of the actual 12 lables
    """ 
    
    ids, inputs, labels = data['id'], data['img'], data['labels']
    # move data to device GPU OR CPU
    inputs, labels = inputs.to(device), labels.to(device)
    return ids, inputs, labels

In [9]:
def getPredictionsFromOutput(outputs):
    """
    We are using BCEWithLogitsLoss for out loss
    In this loss funciton, each label gets the sigmoid (inverse of Logit) before the CE loss
    So our model outputs the raw values on the last FC layer
    This means we have to apply sigmoid to our outputs to squash them between 0 and 1
    We then take values >= .5 as Positive and < .5 as Negative 
    """
    
    predictions = torch.sigmoid(outputs.data) 
    predictions[predictions >= 0.5] = 1 # assign 1 label to those with less than 0.5
    predictions[predictions < 0.5] = 0 # assign 0 label to those with less than 0.5   
    return predictions

In [10]:
def updatePredictionDictionary(dictionary, ids, predictions):
    """
    Keep track of predictions using the same index as our DataFrame
    This will allow us to compare to the actual labels
    
    We only are taking the last prediction for each x-ray, but we could extend this later if wanted.
    """
    
    for i in range(len(ids)):
        id = ids[i].item()    
        dictionary[id] = [int(f.item()) for f in predictions[i]]

In [11]:
def getOverallAccuracy(predictions, labels):
    """
    Overall accuracy for multi-label classifiction will be caculated as
    the number of correct predictions divided by the total number of predctions
    
    The total # of predicitons is the the # of lables times to # in the batch
    i.e. 12 lables with a batch size of 10 = 120 predicitons
    
    The problem with this score is that many of the features have a very low positive rate
    
    This can make this score misleadingly accurate
    """

    batch_size, label_count = labels.shape
    return float((predictions == labels).sum()) / float((batch_size * label_count))   

In [12]:
def processBatch(net, data, optimizer=None):
    """
    Used for both training and validation.
    Validation will not pass in the optimizer.
    """

    # Convert output from loader
    ids, inputs, labels = parseLoaderData(data)
    
    if optimizer:
        # zero the parameter gradients
        optimizer.zero_grad()
        
    # Convert output to predicitons
    outputs = net(inputs)
    predictions = getPredictionsFromOutput(outputs)
    
    return ids, inputs, labels, outputs, predictions 

In [13]:
def backProp(criterion, outputs, labels, optimizer):
    """
    Get loss value from criterion
    run backprop on the loss
    update weights in optimizer
    update epoch loss
    """
    
    loss = criterion(outputs, labels)#.float())
    loss.backward()
    optimizer.step()
    return loss.item()

In [14]:
def getPredictionDataFrame(last_predictions, loaders):
    result = pd.DataFrame(last_predictions).transpose()
    result.columns = loaders.target_columns
    return result

In [15]:
train_accuracy_index = []
train_overall_acc, train_total, train_correct = 0, 0, 0
val_accuracy_index = []
val_overall_acc, val_total, val_correct = 0, 0, 0
losses_index = []

last_training_predictions = {}
last_validation_predictions = {}

In [16]:
learning_rate = 1e-4
num_epochs = 8

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)#, weight_decay=0.9)

In [17]:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    start_time = datetime.now()
    running_loss = 0.0
    epoch_loss = 0
    
    # Training
    for i, data in enumerate(train_loader, 0):
        ids, inputs, labels, outputs, predictions = processBatch(net, data, optimizer)
        updatePredictionDictionary(last_training_predictions, ids, predictions)
        train_overall_acc = getOverallAccuracy(predictions, labels)
        train_accuracy_index.append(train_overall_acc)
        epoch_loss += backProp(criterion, outputs, labels, optimizer)

    epoch_loss = epoch_loss / len(train_loader)
    training_time_elapsed = datetime.now() - start_time
    losses_index.append(epoch_loss)
    
    # Validation
    net.eval()
    with torch.no_grad():
      for data in val_loader:          
            ids, inputs, labels, _, predictions = processBatch(net, data)
            updatePredictionDictionary(last_validation_predictions, ids, predictions)
            val_overall_acc = getOverallAccuracy(predictions, labels)
            val_accuracy_index.append(val_overall_acc)
   
    validation_time_elapsed = datetime.now() - start_time
    
    # stdout Results
    print(f'Epoch [{epoch+1}/{num_epochs}], \
\n          Epoch Loss: {epoch_loss:.4f} \
\n          Training Accuracy: {train_overall_acc:.2%} - (Training time={training_time_elapsed})  \
\n          Validation Accuracy: {val_overall_acc:.2%} - (time={validation_time_elapsed})')

Epoch [1/8], 
          Epoch Loss: 0.9565 
          Training Accuracy: 77.78% - (Training time=0:00:03.182484)  
          Validation Accuracy: 75.00% - (time=0:00:03.590393)
Epoch [2/8], 
          Epoch Loss: 0.7646 
          Training Accuracy: 80.56% - (Training time=0:00:03.163535)  
          Validation Accuracy: 80.00% - (time=0:00:03.574435)
Epoch [3/8], 
          Epoch Loss: 0.7700 
          Training Accuracy: 77.78% - (Training time=0:00:03.168521)  
          Validation Accuracy: 82.22% - (time=0:00:03.574434)
Epoch [4/8], 
          Epoch Loss: 0.8357 
          Training Accuracy: 77.78% - (Training time=0:00:03.162537)  
          Validation Accuracy: 81.11% - (time=0:00:03.581417)
Epoch [5/8], 
          Epoch Loss: 0.8312 
          Training Accuracy: 79.63% - (Training time=0:00:03.183481)  
          Validation Accuracy: 71.11% - (time=0:00:03.592389)
Epoch [6/8], 
          Epoch Loss: 0.7855 
          Training Accuracy: 86.11% - (Training time=0:00:03.151567)  


In [20]:
trained_predictions_df = getPredictionDataFrame(last_training_predictions, loaders)
validation_predictions_df = getPredictionDataFrame(last_validation_predictions, loaders)

In [21]:
display(trained_predictions_df.loc[45510, :])
display(loaders.train_df[loaders.target_columns].loc[45510, :])

Enlarged_Cardiomediastinum    0
Cardiomegaly                  0
Lung_Opacity                  0
Lung_Lesion                   0
Edema                         0
Consolidation                 0
Pneumonia                     0
Atelectasis                   0
Pneumothorax                  0
Pleural_Effusion              1
Pleural_Other                 0
Fracture                      0
Name: 45510, dtype: int64

Enlarged_Cardiomediastinum    0
Cardiomegaly                 -1
Lung_Opacity                  0
Lung_Lesion                   0
Edema                         0
Consolidation                 0
Pneumonia                     0
Atelectasis                   0
Pneumothorax                  0
Pleural_Effusion              1
Pleural_Other                 0
Fracture                      0
Name: 45510, dtype: int8